In [ ]:
% pip install weaviate-client openai

In [4]:
import sys
sys.path.append('../packages/code_langchain/code_langchain')
from chain import chain

## Check

Invoke the chain

In [ ]:
question = "how to chat PDF with chroma? Use LCEL"
answer = chain.invoke({"question": question})

In [3]:
answer

'Here is a combined code snippet that demonstrates how to chat with a PDF using Chroma and LangChain Expression Language (LCEL):\n\n```python\nfrom langchain.document_loaders import PyPDFLoader\nfrom langchain.vectorstores import Chroma\nfrom langchain.retrievers.self_query.base import SelfQueryRetriever\nfrom langchain.chains import RetrievalQA\nfrom langchain.llms import OpenAI\nfrom langchain.embeddings import YourEmbeddingFunction\n\n# Step 1: Load PDF content\nloader = PyPDFLoader("path/to/pdf/file.pdf")\npages = loader.load_and_split()\n\n# Step 2: Create an embedding function\nembedding_function = YourEmbeddingFunction()\n\n# Step 3: Load PDF pages into Chroma\nvectorstore = Chroma.from_documents(pages, embedding_function)\n\n# Step 4: Create a SelfQueryRetriever\nretriever = SelfQueryRetriever.from_vectorstore(vectorstore)\n\n# Step 5: Define the question you want to query\nquestion = "Your question here"\n\n# Step 6: Use LCEL to construct the query expression\nquery_expression

# Eval

Run eval, using `eval.csv` to generate a dataset `code-langchain-eval`.

In [5]:
import uuid
from langchain.smith import RunEvalConfig
from langsmith.client import Client

# Config
client = Client()
eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

# Experiments
chain_map = {
    "code-langchain-v1": final_answer_chain,
}

# Run evaluation
run_id = uuid.uuid4().hex[:4]
test_runs = {}
for project_name, chain in chain_map.items():
    test_runs[project_name] = client.run_on_dataset(
        dataset_name="code-langchain-eval",
        llm_or_chain_factory=chain,
        evaluation=eval_config,
        verbose=True,
        project_name=f"{run_id}-{project_name}",
        project_metadata={"chain": project_name},
    )

View the evaluation results for project '877f-code-langchain-v1' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/5dcb1814-37ec-4e0a-8b8d-be07aa2b0037/compare?selectedSessions=c08abee2-e005-44bd-a696-b5a9fdce318f

View all tests for Dataset code-langchain-eval at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/5dcb1814-37ec-4e0a-8b8d-be07aa2b0037
[>                                                 ] 0/5

/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:454: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)
/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:454: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See 

[--------->                                        ] 1/5

/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:454: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[------------------->                              ] 2/5

/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:454: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[--------------------------------------->          ] 4/5

/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:454: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[------------------------------------------------->] 5/5

,output,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,5,5.000000,0,5.000000,5
unique,5,NaN,0,NaN,5
top,"Sure, based on the answers to the subquestions...",NaN,NaN,NaN,f0201b9e-8123-4bf8-b0ee-64683995251f
freq,1,NaN,NaN,NaN,1
mean,NaN,0.200000,NaN,69.337693,NaN
std,NaN,0.447214,NaN,10.352142,NaN
min,NaN,0.000000,NaN,59.624474,NaN
25%,NaN,0.000000,NaN,60.368564,NaN
50%,NaN,0.000000,NaN,69.543329,NaN
75%,NaN,0.000000,NaN,72.128616,NaN
